In [842]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.impute import SimpleImputer
from datetime import datetime

In [843]:
df=pd.read_csv("Marvel_Comics.csv")
df.head()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free
1,A Year of Marvels: August Infinite Comic (2016),(2016),A Year of Marvels: August Infinite Comic (2016...,"August 10, 2016","It’s August, and Nick Fury is just in time to ...",Jamal Campbell,"Chris Sims, Chad Bowers",NaN,Marvel Universe,Infinite Comic,NaN,Free
2,A Year of Marvels: February Infinite Comic (2016),(2016),A Year of Marvels: February Infinite Comic (20...,"February 10, 2016",Join us in a brand new Marvel comics adventure...,"Danilo S. Beyruth, M Mast",Ryan North,NaN,Marvel Universe,Infinite Comic,Rated T+,Free
3,A Year of Marvels: July Infinite Comic (2016),(2016),A Year of Marvels: July Infinite Comic (2016) #1,"June 29, 2016",Celebrating the Fourth of July is complicated ...,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free
4,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free


In [844]:
df.shape

(34992, 12)

In [845]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34466 non-null  object
 3   publish_date       34466 non-null  object
 4   issue_description  30395 non-null  object
 5   penciler           25482 non-null  object
 6   writer             27595 non-null  object
 7   cover_artist       12255 non-null  object
 8   Imprint            11684 non-null  object
 9   Format             32894 non-null  object
 10  Rating             12619 non-null  object
 11  Price              32894 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [846]:
df.describe()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
count,34992,34992,34466,34466,30395,25482,27595,12255,11684,32894,12619,32894
unique,4935,412,33757,3317,29331,3915,3082,1010,39,11,36,53
top,Uncanny X-Men (1981 - 2011),(1968 - 1996),X-Men: The Complete Age of Apocalypse Epic Boo...,"November 30, -0001",Please note that these digital editions collec...,Sal Buscema,Brian Michael Bendis,Gil Kane,Marvel Universe,Comic,Rated T+,Free
freq,588,718,10,543,37,394,859,217,7786,31899,3204,15136


## FASE 1 Diseño de la base de datos 

In [847]:
client = MongoClient("mongodb://localhost:27017")
db = client['marvel']

collections = ["raw_marvel", "curated_marvel", "analytics_marvel"]

for coll in collections:
    if coll not in db.list_collection_names():
        db.create_collection(coll)
        print(f"Colección '{coll}' creada")
    else:
        print(f"Colección '{coll}' ya existe")



Colección 'raw_marvel' ya existe
Colección 'curated_marvel' ya existe
Colección 'analytics_marvel' ya existe


## FASE 2 Carga de datos en RAW

In [848]:
df_raw=df.copy()

In [849]:
df_raw.drop(columns=["issue_description","issue_title"],inplace=True)

In [850]:
#EXTRAEMOS LOS AÑOS DE LOS PARÉNTESIS Y LOS CONVERTIMOS A ENTEROS
df_raw["release_year"] = (df_raw["active_years"].str.extract(r"\((\d{4})")[0].astype("int64"))


In [851]:
#NORMALIZACIÓN DE NOMBRES DE COLUMNAS
cols=df_raw.columns.astype(str).str.title()
print(cols)

Index(['Comic_Name', 'Active_Years', 'Publish_Date', 'Penciler', 'Writer',
       'Cover_Artist', 'Imprint', 'Format', 'Rating', 'Price', 'Release_Year'],
      dtype='object')


In [852]:
#CAMBIAMOS LOS PRECIOS A FLOAT
df_raw["Price"] = df_raw["Price"].str.strip().replace("Free", "0")  # Free → 0
df_raw["Price"] = df_raw["Price"].str.replace(r"[\$,]", "", regex=True)  # quitar $ y comas
df_raw["Price"] = df_raw["Price"].astype("float64")

In [853]:
#CAMBIAMOS EL FORMATO DE FECHA
df_raw["publish_date"] = pd.to_datetime(df_raw["publish_date"], errors="coerce")
datetime_cols = df_raw.select_dtypes(include=["datetime64[ns]"]).columns
df_raw[datetime_cols] = df_raw[datetime_cols].astype("object")

#IMPUTAMOS
imputer = SimpleImputer(strategy="most_frequent")
df_raw[datetime_cols] = imputer.fit_transform(df_raw[datetime_cols])
df_raw[datetime_cols] = df_raw[datetime_cols].apply(pd.to_datetime, errors="coerce")
print(df_raw[datetime_cols].isna().sum())


publish_date    0
dtype: int64


In [854]:
df_raw.dtypes

comic_name              object
active_years            object
publish_date    datetime64[ns]
penciler                object
writer                  object
cover_artist            object
Imprint                 object
Format                  object
Rating                  object
Price                  float64
release_year             int64
dtype: object

In [855]:
#CAMBIAMOS LOS NAN DE STRING A NULL
obj_cols = df_raw.select_dtypes(include="O").columns

df_raw[obj_cols] = df_raw[obj_cols].where(df_raw[obj_cols].notna(), None)

In [856]:
df_raw.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year
0,A Year of Marvels: April Infinite Comic (2016),(2016),2016-04-01,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,0.0,2016
1,A Year of Marvels: August Infinite Comic (2016),(2016),2016-08-10,Jamal Campbell,"Chris Sims, Chad Bowers",None,Marvel Universe,Infinite Comic,None,0.0,2016
2,A Year of Marvels: February Infinite Comic (2016),(2016),2016-02-10,"Danilo S. Beyruth, M Mast",Ryan North,None,Marvel Universe,Infinite Comic,Rated T+,0.0,2016
3,A Year of Marvels: July Infinite Comic (2016),(2016),2016-06-29,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0,2016
4,A Year of Marvels: June Infinite Comic (2016),(2016),2016-06-15,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0,2016


In [857]:
print(df_raw.isna().sum())

comic_name          0
active_years        0
publish_date        0
penciler         9510
writer           7397
cover_artist    22737
Imprint         23308
Format           2098
Rating          22373
Price            2098
release_year        0
dtype: int64


In [858]:
print(df_raw.isna().sum()/len(df_raw))

comic_name      0.000000
active_years    0.000000
publish_date    0.000000
penciler        0.271776
writer          0.211391
cover_artist    0.649777
Imprint         0.666095
Format          0.059957
Rating          0.639375
Price           0.059957
release_year    0.000000
dtype: float64


In [859]:
#VAMOS A COMPROBAR QUE CATEGORIAS DE RATING HAY
unique_values = df_raw["Rating"].unique()
print(unique_values)



[' Rated T+' None ' Rated T' ' ALL AGES' ' A' ' Parental Advisory'
 ' Marvel Psr' ' No Rating' ' MARVEL PSR' ' T' ' RATED T' ' Max'
 ' RATED T+' ' RATED A' ' All Ages' ' T+' ' Rated a' ' Rated A'
 ' Parental Advisory/Explicit Content' ' PARENTAL SUPERVISION'
 ' PARENTAL ADVISORY' ' Mature' ' MARVEL PSR+' ' EXPLICIT CONTENT'
 ' PARENTAL ADVISORYSLC' ' Parental AdvisorySLC' ' Parental Advisoryslc'
 ' Explicit Content' ' PARENTAL ADVISORY/EXPLICIT CONTENT' ' NO RATING'
 ' NOT IN ORACLE' ' Parental Guidance' ' Ages 10 & Up' ' Not in Oracle'
 ' MAX' ' Marvel Psr+' ' Ages 9+']


In [860]:
# UNIFICAMOS LAS CATEGORÍAS DE RATING (Manteniendo nulos)
df_raw["Rating_clean"] = df_raw["Rating"].str.lower().str.strip()

rating_map = {
    "t+": "T+",
    "rated t+": "T+",
    "rated t": "T",
    "t": "T",
    "all ages": "All Ages",
    "a": "A",
    "rated a": "A",
    "parental advisory": "Parental Advisory",
    "parental advisory/explicit content": "Parental Advisory",
    "parental advisoryslc": "Parental Advisory",
    "explicit content": "Explicit Content",
    "marvel psr": "Marvel PSR",
    "marvel psr+": "Marvel PSR+",
    "max": "Max",
    "mature": "Mature",
    "parental guidance": "Parental Advisory",
    "parental supervision": "Parental Advisory",
    "ages 10 & up": "All Ages",
    "ages 9+": "All Ages"
}

df_raw["Rating"] = df_raw["Rating_clean"].replace(rating_map)

df_raw.drop(columns=["Rating_clean"], inplace=True)

print(f"Nulos en Rating: {df_raw['Rating'].isna().sum()}")
print(df_raw["Rating"].value_counts(dropna=False))

Nulos en Rating: 22373
Rating
None                 22373
T+                    4410
T                     3057
Parental Advisory     1613
All Ages              1199
Marvel PSR            1060
A                      903
Explicit Content       149
no rating              102
Mature                  44
Max                     34
not in oracle           27
Marvel PSR+             21
Name: count, dtype: int64


In [861]:
#DUPLICADOS
num_duplicates = df_raw.duplicated().sum()
print(f"Hay {num_duplicates} registros duplicados")


Hay 771 registros duplicados


In [862]:
#ELIMINAMOS LOS DUPLICADOS
df_raw = df_raw.drop_duplicates()



In [863]:
#AÑADIMOS UN ID
df_raw["id"] = range(1, len(df_raw) + 1)


In [864]:
coleccion = db["raw_marvel"] 

In [865]:
coleccion.delete_many({})
print("Colección 'raw_marvel' limpia")

Colección 'raw_marvel' limpia


In [866]:
#INSERCIÓN EN LA BASE DE DATOS
data_to_insert = df_raw.to_dict(orient='records')

result = coleccion.insert_many(data_to_insert)

print(f"Se insertaron {len(result.inserted_ids)} documentos en 'raw_marvel'")


Se insertaron 34221 documentos en 'raw_marvel'


## FASE 3 CRUD completo sobre RAW

In [867]:
#CREAMOS UN COMIC
nuevo_comic = {
    "comic_name": "Avengers: Future Fight (2026)",
    "active_years": 2026,
    "publish_date": datetime(2026, 3, 1),
    "penciler": "Emily White",
    "writer": "Mark Black",
    "cover_artist": "Chris Sanders",
    "Imprint": "Marvel Universe",
    "Format": "Graphic Novel",
    "Rating": "T+",
    "Price": 5.99,
    "id": 34223
}


### INSERCIÓN

In [868]:
#INSERTAMOS UN COMIC
result_create = coleccion.insert_one(nuevo_comic)
print(f"Documento insertado con _id: {result_create.inserted_id}")

Documento insertado con _id: 699877828a92e02a52f23593


### FIND

In [869]:
#ENCONTRAMOS CON FILTRO DE CAMPO CATEGÓRICO
all_ages_comics = pd.DataFrame(coleccion.find({"Rating": "All Ages"}))
print("Dataframe de comics para todas las edades:")
all_ages_comics.head()

Dataframe de comics para todas las edades:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year,id
0,699877828a92e02a52f1b015,A-Next (1998 - 1999),(1998 - 1999),2006-08-09,None,None,None,MARVEL UNIVERSE,Digest,All Ages,7.99,1998,48
1,699877828a92e02a52f1b346,Amazing Fantasy Facsimile Edition (2019),(2019),2019-10-09,None,None,None,Marvel Universe,Comic,All Ages,3.99,2019,865
2,699877828a92e02a52f1b48e,Amazing Spider-Man Annual (1964 - 2018),(1964 - 2018),2005-04-01,None,None,None,MARVEL UNIVERSE,Trade Paperback,All Ages,0.00,1964,1193
3,699877828a92e02a52f1b490,Amazing Spider-Man Annual (1964 - 2018),(1964 - 2018),2004-01-01,None,None,None,MARVEL UNIVERSE,Hardcover,All Ages,0.00,1964,1195
4,699877828a92e02a52f1b6c0,Avengers (1963 - 1996),(1963 - 1996),2005-04-01,None,None,None,MARVEL UNIVERSE,Trade Paperback,All Ages,0.00,1963,1755


In [870]:
#FILTRO DE COMPARACIÓN NUMÉRICA DE PRECIO
comics_con_precio = pd.DataFrame(coleccion.find({"Price": {"$gt": 5.00}}))
print('Dataframe con los comics con precio mayor a 5 dólares:')
comics_con_precio.head()

Dataframe con los comics con precio mayor a 5 dólares:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year,id
0,699877828a92e02a52f1b015,A-Next (1998 - 1999),(1998 - 1999),2006-08-09,None,None,None,MARVEL UNIVERSE,Digest,All Ages,7.99,1998.0,48
1,699877828a92e02a52f1b026,Absolute Carnage (2019),(2019),2019-08-07,Ryan Stegman,Donny Cates,Ryan Stegman,None,Comic,T+,7.99,2019.0,65
2,699877828a92e02a52f1b189,All-New Marvel Now! Point One (2014),(2014),2014-01-08,Steve Mcniven,"G. Willow Wilson, Al Ewing, James Robinson, Na...",None,Marvel Universe,Comic,None,5.99,2014.0,420
3,699877828a92e02a52f1b220,All-New X-Men Annual (2014),(2014),2014-12-24,None,None,None,None,Comic,None,24.99,2014.0,571
4,699877828a92e02a52f1b232,"All-New, All-Different Avengers (2015 - 2016)",(2015 - 2016),2015-10-07,Kenneth Rocafort,"Gerry Duggan, Mark Waid, James Robinson, G. Wi...",None,None,Comic,None,5.99,2015.0,589


In [871]:
#PROYECCIÓN DE CAMPOS
projection_comics =pd.DataFrame(coleccion.find({}, {"comic_name": 1, "Price": 1, "_id": 0}))
print('Dataframe de comis con solo las columnas nombre y precio:')
projection_comics.head()

Dataframe de comis con solo las columnas nombre y precio:


,comic_name,Price
0,A Year of Marvels: April Infinite Comic (2016),0.0
1,A Year of Marvels: August Infinite Comic (2016),0.0
2,A Year of Marvels: February Infinite Comic (2016),0.0
3,A Year of Marvels: July Infinite Comic (2016),0.0
4,A Year of Marvels: June Infinite Comic (2016),0.0


In [872]:
#ORDENACION Y LÍMITE DEL CAMPO PRECIO LIMITADO A 3
df_top_priced = pd.DataFrame(coleccion.find().sort("Price", -1).limit(3))
print('Dataframe de top 3 comics más caros:')
df_top_priced.head()

Dataframe de top 3 comics más caros:


,_id,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year,id
0,699877828a92e02a52f1dc5d,Howard the Duck Annual (1977),(1977),2008-07-23,None,None,None,MARVEL UNIVERSE,Hardcover,T+,99.99,1977,11384
1,699877828a92e02a52f1dc17,Howard the Duck (1976 - 1979),(1976 - 1979),2008-07-23,None,None,None,MARVEL UNIVERSE,Hardcover,T+,99.99,1976,11314
2,699877828a92e02a52f1c947,Daring Mystery Comics (1940 - 1942),(1940 - 1942),2008-01-23,None,None,None,MARVEL UNIVERSE,Hardcover,A,59.99,1940,6498


### UPDATE

In [873]:
coleccion.update_one(
    {"id": 1},         
    {"$set": {"Rating": "All Ages"}} 
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [874]:
coleccion.update_many(
    {},  
    [{"$set": {"Rating": {"$toUpper": "$Rating"}}}] 
)

UpdateResult({'n': 34222, 'nModified': 25881, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### DELETE

In [875]:
coleccion.delete_one({"id": 1})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [876]:
coleccion.delete_many({"Price": 0})

DeleteResult({'n': 14470, 'ok': 1.0}, acknowledged=True)

## FASE 4 CURATED (limpieza y transformación)

In [877]:
df_cur = pd.DataFrame(list(db['raw_marvel'].find()))
df_cur.drop(columns=['_id'], inplace=True)

### GESTIÓN DE NULOS

In [878]:
print(df_cur.isna().sum()/len(df_cur))

comic_name      0.000000
active_years    0.000000
publish_date    0.000000
penciler        0.259177
writer          0.192142
cover_artist    0.579363
Imprint         0.566098
Format          0.103792
Rating          0.000000
Price           0.103792
release_year    0.000051
id              0.000000
dtype: float64


In [879]:
#IMPUTAMOS LOS NULOS DE PRECIO A LA MEDIANA
df_cur['Price']= df_cur['Price'].fillna(df_cur['Price'].median())

In [880]:
#IMPUTAMOS EL RATING A NO RATED
df_cur['Rating'] = df_cur['Rating'].fillna('Not Rated')
df_cur.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year,id
0,A Year of Marvels: The Amazing (2016),(2016),2016-04-27,"Danilo S. Beyruth, Ryan Browne","Ryan North, Amy Chu",Jamal Campbell,None,Comic,,4.99,2016.0,11
1,A Year of Marvels: The Incredible (2016),(2016),2016-06-15,Leonardo Romero,Dennis Culver,Jamal Campbell,None,Comic,,4.99,2016.0,12
2,A Year of Marvels: The Uncanny (2016),(2016),2016-12-07,None,None,None,Marvel Universe,Comic,T+,4.99,2016.0,13
3,A+X (2012 - 2014),(2012 - 2014),2014-03-26,"Matteo Lolli, David Yardin, Will Sliney","Gerry Duggan, Jim Krueger",Kevin Nowlan,None,Comic,,3.99,2012.0,14
4,A+X (2012 - 2014),(2012 - 2014),2014-02-19,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,None,Comic,T,3.99,2012.0,15


In [881]:
#IMPUTAMOS LOS NULOS DE IMPRENTA Y FORMATO
#LOS DOS SE IMPUTAN AL VALOR MAS FRECUENTE AL SER POCOS NULOS

cols_to_fix = ['Imprint', 'Format']
for col in cols_to_fix:
    df_cur[col] = df_cur[col].astype(str).str.strip().str.upper()
    df_cur[col] = df_cur[col].replace(['NAN', 'NONE', 'NULL', ''], np.nan)

imputer = SimpleImputer(strategy='most_frequent')

df_cur[cols_to_fix] = imputer.fit_transform(df_cur[cols_to_fix])


In [882]:
#IMPUTAMOS LOS TIPOS DE ARTISTAS EN EL VALOR UNKNOWN
author_cols = ['penciler', 'writer', 'cover_artist']

for col in author_cols:
    df_cur[col] = df_cur[col].astype(str).str.strip().replace(['nan', 'None', 'NULL', ''], np.nan)

from sklearn.impute import SimpleImputer
imputer_author = SimpleImputer(strategy='constant', fill_value='Unknown')

df_cur[author_cols] = imputer_author.fit_transform(df_cur[author_cols])


In [883]:

print(df_cur.isna().sum()/len(df_cur))

comic_name      0.000000
active_years    0.000000
publish_date    0.000000
penciler        0.000000
writer          0.000000
cover_artist    0.000000
Imprint         0.000000
Format          0.000000
Rating          0.000000
Price           0.000000
release_year    0.000051
id              0.000000
dtype: float64


### CORRECCIÓN DE TIPOS

In [884]:
cat_cols=['Rating', 'Format','Imprint']
df_cur[cat_cols]=df_cur[cat_cols].astype('category')
df_cur.dtypes

comic_name              object
active_years            object
publish_date    datetime64[ns]
penciler                object
writer                  object
cover_artist            object
Imprint               category
Format                category
Rating                category
Price                  float64
release_year           float64
id                       int64
dtype: object

### CREACION DE 2 COLUMNAS

In [885]:
#COLUMNA DE AÑOS ACTIVOS CONTANDO SUS AÑOS DESDE LANZAMIENTO
current_year = datetime.now().year
df_cur["active_years"] = current_year - df_cur["release_year"]

In [886]:
#COLUMNA DE DÉCADA
df_cur["decade"] = (df_cur["release_year"] // 10 * 10).astype("Int64").astype("category")

In [887]:
#df_cur.columns = df_cur.columns.str.title()
df_cur.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price,release_year,id,decade
0,A Year of Marvels: The Amazing (2016),10.0,2016-04-27,"Danilo S. Beyruth, Ryan Browne","Ryan North, Amy Chu",Jamal Campbell,MARVEL UNIVERSE,COMIC,,4.99,2016.0,11,2010
1,A Year of Marvels: The Incredible (2016),10.0,2016-06-15,Leonardo Romero,Dennis Culver,Jamal Campbell,MARVEL UNIVERSE,COMIC,,4.99,2016.0,12,2010
2,A Year of Marvels: The Uncanny (2016),10.0,2016-12-07,Unknown,Unknown,Unknown,MARVEL UNIVERSE,COMIC,T+,4.99,2016.0,13,2010
3,A+X (2012 - 2014),14.0,2014-03-26,"Matteo Lolli, David Yardin, Will Sliney","Gerry Duggan, Jim Krueger",Kevin Nowlan,MARVEL UNIVERSE,COMIC,,3.99,2012.0,14,2010
4,A+X (2012 - 2014),14.0,2014-02-19,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,MARVEL UNIVERSE,COMIC,T,3.99,2012.0,15,2010


In [888]:
db.curated_marvel.delete_many({})

DeleteResult({'n': 19751, 'ok': 1.0}, acknowledged=True)

In [889]:
#INSERCIÓN EN LA BASE DE DATOS
df_to_insert = df_cur.copy()

df_to_insert["creators"] = (
    df_to_insert[["writer", "penciler", "cover_artist"]]
    .to_dict(orient="records")
)
df_to_insert.rename(columns={"_Id": "_id"}, inplace=True)
df_to_insert.drop(columns=["writer", "penciler", "cover_artist"], inplace=True)
records = df_to_insert.to_dict(orient="records")
db.curated_marvel.insert_many(records)

InsertManyResult([ObjectId('699877848a92e02a52f23594'), ObjectId('699877848a92e02a52f23595'), ObjectId('699877848a92e02a52f23596'), ObjectId('699877848a92e02a52f23597'), ObjectId('699877848a92e02a52f23598'), ObjectId('699877848a92e02a52f23599'), ObjectId('699877848a92e02a52f2359a'), ObjectId('699877848a92e02a52f2359b'), ObjectId('699877848a92e02a52f2359c'), ObjectId('699877848a92e02a52f2359d'), ObjectId('699877848a92e02a52f2359e'), ObjectId('699877848a92e02a52f2359f'), ObjectId('699877848a92e02a52f235a0'), ObjectId('699877848a92e02a52f235a1'), ObjectId('699877848a92e02a52f235a2'), ObjectId('699877848a92e02a52f235a3'), ObjectId('699877848a92e02a52f235a4'), ObjectId('699877848a92e02a52f235a5'), ObjectId('699877848a92e02a52f235a6'), ObjectId('699877848a92e02a52f235a7'), ObjectId('699877848a92e02a52f235a8'), ObjectId('699877848a92e02a52f235a9'), ObjectId('699877848a92e02a52f235aa'), ObjectId('699877848a92e02a52f235ab'), ObjectId('699877848a92e02a52f235ac'), ObjectId('699877848a92e02a52f235

## FASE 5 ANALYTICS (agregaciones)

## FASE 6 Rendimiento y diseño

## FASE 7 Visualización y conclusiones 